In [2]:
class Address:
    def __init__(self, ip, port):
        self.ip = ip
        self.port = port
    def __hash__(self):
        return hash((self.ip, self.port))
    def __lt__(self, rhs):
        return self.ip < rhs.ip or (self.ip == rhs.ip and self.port < rhs.port)
    def __eq__(self, rhs):
        return not (self < rhs or rhs < self)
    def __str__(self):
        return f"{self.ip:>15}:{self.port:<5}"

class Flow:
    def __init__(self, pkt):
        self.src = Address(pkt['IP'].src, pkt['IP'].sport)
        self.dst = Address(pkt['IP'].dst, pkt['IP'].dport)
    def __hash__(self):
        return hash(self.ordered)
    def __eq__(self, rhs):
        return self.ordered == rhs.ordered
    def __lt__(self, rhs):
        return (
            self.src[0] < rhs.src[0] or
            (self.src[0] == rhs.src[0] and self.src[1] < rhs.src[1]) or
            (self.src == rhs.src and
                (self.dst[0] < rhs.dst[0] or
                (self.dst[0] == rhs.dst[0] and self.dst[1] < rhs.dst[1])))
        )
    def __str__(self):
        return f"{self.src} ==> {self.dst}"
    @property
    def ordered(self):
        return (self.src, self.dst) if self.src < self.dst else (self.dst, self.src)
    @property
    def forward(self):
        return self
    @property
    def reverse(self):
        return self.__class__(self.dst, self.src)

In [4]:
IPv4 = namedtuple('IPv4', 'ver_ihl tos len id offset ttl proto csum src dst')

FLAGS = {
	"F":0x1, "S":0x2, "R":0x4, "P":0x8, "A":0x10, "U":0x20, "E":0x40, "C":0x80,
#	0x1:"F", 0x2:"S", 0x4:"R", 0x8:"P", 0x10:"A", 0x20:"U", 0x40:"E", 0x80:"C"
}
FLAGS_ORDER = "SAFCR"

CLIENT_STATES = {
    'CLOSED': {
        'S': 'SYN_SENT',
    },
    'SYN_SENT': {
        'SA': 'ESTABLISHED',
        # deal with timeout to go back to CLOSED
    },
    'ESTABLISHED': {
        'F': 'FIN_WAIT_1', # (FIN comes from client)
    },
    'FIN_WAIT_1' : {
        'A': 'FIN_WAIT_2', # passive close, client sends nothing
        'F': 'CLOSING', # simultaneous close, client sends ACK
        'FA': 'TIME_WAIT', # FIN and ACK in one packet, same as simultaneous close, client sends ACK
    },
    'CLOSING': {
        'A': 'TIME_WAIT', # client sends nothing
    },
    'FIN_WAIT_2': {
        'F': 'TIME_WAIT', # could wait for timeout, but all transitions to this state lead to CLOSED
    },
}

SERVER_STATES = {
    'LISTEN': {
        'S' : 'SYN_RCVD', # SYN
    },
    'SYN_RCVD': {
        'A': 'ESTABLISHED', # ACK
        'R': 'LISTEN', # RESET
        'F': 'FIN_WAIT_1' # active close
    },
    'ESTABLISHED': {
        'F': 'CLOSE_WAIT', # server sends ACK
    },
    'CLOSE_WAIT': {
        'F': 'LAST_ACK', # (FIN comes from server) server sends FIN
    },
    'LAST_ACK': {
        'A': 'CLOSED',
    }
}

is_syn_pkt = lambda pkt: 'TCP' in pkt and pkt['TCP'].flags == FLAGS['S']
is_synack_pkt = lambda pkt: 'TCP' in pkt and pkt['TCP'].flags == (FLAGS['S'] | FLAGS['A'])
#build_flags = lambda s: sum([FLAGS[f] for f in s])

class TCPSession:
    def __init__(self, pkt=None):

        if not 'TCP' in pkt:
            raise Exception("Not a TCP Packet")
        if not is_syn_pkt(pkt):
            raise Exception("Not valid SYN")

        self.flow = Flow(pkt) # client => server
        self.packets = []

        # 0 is now, 1 is the future Flags
        self.server_state = "LISTEN"
        self.client_state = "CLOSED"

        if pkt is None:
            return

        self.handle_packet(pkt)
#       self.server_close_time = -1.0
#       self.client_close_time = -1.0
#       self.fin_wait_time = -1.0

    @property
    def client(self):
        return self.flow.src
    @property
    def server(self):
        return self.flow.dst

    def add(self, pkt):
        if not 'TCP' in pkt:
            raise Exception("Not a TCP Packet")

        # determine in what context we are handling this packet
#        flow = Flow(pkt)
#        if flow.forward != self.flow and flow.reverse != self.flow:
#            raise Exception("Not a valid packet for this model")

        self.handle_packet(pkt)
#		if flow.dst == self.server:
#			v =  self.add_client_pkt(pkt)
#			if self.is_fin_wait():
#				self.fin_wait_time = pkt.time
#			return v
#		else:
#			v = self.add_server_pkt(pkt)
#			if self.is_fin_wait():
#				self.fin_wait_time = pkt.time
#			return v
#		raise Exception("Not a valid packet for this model")

    def handle_packet(self, pkt):
        flags = ''.join(f for f,x in FLAGS.items() if x & pkt['TCP'].flags)

        print(Flow(pkt), flags, len(pkt.payload))
#        flags = pkt['TCP'].flags

        flags = ''.join(f for f in FLAGS_ORDER if f in flags)

        self.client_state = CLIENT_STATES[self.client_state].get(flags) or self.client_state
        self.server_state = SERVER_STATES[self.server_state].get(flags) or self.server_state
        print(self.client_state, self.server_state)
        
        self.packets.append(pkt)
        
        


NameError: name 'namedtuple' is not defined

In [ ]:
class TCPSessionExtractor:
    def __init__(self, filename, count=-1):
        from scapy.all import rdpcap
        self.packets = scapy.utils.rdpcap(filename, count=count)
        self.sessions = {}
        
        for pkt in self.packets:
            if not 'IP' in pkt:
                continue

            flow = Flow(pkt)

            if flow not in self.sessions and is_syn_pkt(pkt):
                self.sessions[flow] = TCPSession(pkt)
            elif flow in self.sessions:
                self.sessions[flow].add(pkt)
                

In [ ]:

filename = '../data/pcap/tipc.pcap'
extractor = TCPSessionExtractor(filename)#, 100)

#print(extractor.packets[0].getlayer())

#for pkt in extractor.packets:
#    print("trying")


   192.168.0.11:42488 ==>    192.168.0.12:666   S 60
SYN_SENT SYN_RCVD
   192.168.0.12:666   ==>    192.168.0.11:42488 SA 60
ESTABLISHED SYN_RCVD
   192.168.0.11:42488 ==>    192.168.0.12:666   A 52
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   PA 92
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 A 52
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 PA 92
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   A 52
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 PA 108
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   A 52
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   PA 108
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 PA 108
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   PA 108
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 PA 108
ESTABLISHED ESTABLI

   192.168.0.12:666   ==>    192.168.0.11:42488 PA 108
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   A 52
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 PA 108
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   A 52
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 PA 108
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   A 52
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   PA 108
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 A 52
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   PA 120
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 A 52
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   PA 120
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 A 52
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   PA 108
ESTABLISH

   192.168.0.11:42488 ==>    192.168.0.12:666   A 52
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   PA 108
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 A 52
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 PA 99
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   A 52
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 PA 108
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   A 52
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   PA 99
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 A 52
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   PA 92
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 A 52
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   PA 120
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 A 52
ESTABLISHED ES

   192.168.0.11:42488 ==>    192.168.0.12:666   PA 120
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 A 52
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   PA 108
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 A 52
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 PA 99
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   A 52
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 PA 160
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   A 52
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 PA 108
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   A 52
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   PA 108
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 PA 108
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   A 52
ESTABLISHE

   192.168.0.11:42488 ==>    192.168.0.12:666   PA 92
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 A 52
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   PA 108
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 PA 108
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   A 52
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 PA 108
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   A 52
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   PA 108
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 A 52
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 PA 108
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   A 52
ESTABLISHED ESTABLISHED
   192.168.0.12:666   ==>    192.168.0.11:42488 PA 108
ESTABLISHED ESTABLISHED
   192.168.0.11:42488 ==>    192.168.0.12:666   A 52
ESTABLISHE

In [ ]:
def checksum(pkt):
    IPv4 = namedtuple('IPv4', 'ver_ihl tos len id offset ttl proto csum src dst')
    ip = IPv4._make(struct.unpack("!BBHHHBBHII", bytes(pkt['IP'])[:20]))
    ip_header_sum = ip.src + ip.dst + ip.proto + ip.len

    csum = ip_header_sum
    return csum

for pkt in extractor.packets:
    print(checksum(pkt))

TypeError: can only concatenate str (not "int") to str

In [ ]:
dir(extractor.packets[0]['IP'])

['_PickleType',
 '__all_slots__',
 '__annotations__',
 '__bool__',
 '__bytes__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__iterlen__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__orig_bases__',
 '__parameters__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__rtruediv__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '_answered',
 '_defrag_pos',
 '_do_summary',
 '_is_protocol',
 '_name',
 '_overload_fields',
 '_pkt',
 '_resolve_alias',
 '_show_or_dump',
 '_superdir',
 '_ttl',
 'add_payload',
 'add_underlayer',
 'aliastypes',
 'answe

In [ ]:
print(extractor.packets[0]['IP'][0])

b'E\x00\x00<\xb8>@\x00?\x06\x02\x16\xc0\xa8\x00\x0b\xc0\xa8\x00\x0c\xa5\xf8\x02\x9an\x17\\\x97\x00\x00\x00\x00\xa0\x02\x16\xd0W\xe2\x00\x00\x02\x04\x05\xb4\x04\x02\x08\n\xff\xff\xe4\xa9\x00\x00\x00\x00\x01\x03\x03\x02'


In [ ]:
import struct
from collections import namedtuple

pkt = extractor.packets[0]['IP']
IPv4 = namedtuple('IPv4', 'ver_ihl tos len id offset ttl proto csum src dst')
ip = IPv4._make(struct.unpack("!BBHHHBBHII", bytes(pkt)[:20]))
ip

IPv4(ver_ihl=69, tos=0, len=60, id=47166, offset=16384, ttl=63, proto=6, csum=534, src=3232235531, dst=3232235532)